# **Presentación**


## **Tecnicatura Universitaria en Inteligencia Artificial**
### *Trabajo Práctico Integrador*
### ***Aprendizaje Automatico 1***
---

**Equipo**:
- Ferrucci Constantino
- Giampaoli Fabio

<br>

**Fechas de entrega**:
- 27/10/2023 (segunda entrega hasta item 4)


---

## **Resumen**


Esta es la segunda parte del proyecto de la asignatura Aprendizaje Automatico 1. Puede consultar la parte 1 del mismo en el repositorio de [github](https://github.com/Shannon-21/Aprendizaje_Automatico_1.git), o bien el siguiente [notebook de Google Colab](https://colab.research.google.com/drive/1P32i9TmMZgLiKAblI0vKwltg7GMwnqvw?usp=sharing).\
<br>

La primera parte consiste en la limpieza del dataset ofrecido y obtencion de predicciones y metricas de las mismas sobre la cantidad de lluvia en las ciudades de Australia elegidas para el dia posterior al ingresado.\
<br>

El dataset elegido consiste de variables meteorologicas de Australia en determinado periodo de tiempo que seran de interes para la predicion de la presencia de lluvia en algun dia con ciertas caracteristicas mediante modelos estudiados en el cursado de la materia.\
<br>

En esta etapa del trabajo practico, es de principal interes el analisis y tratamiento del conjunto de datos para conocer el contexto sobre el cual vamos a desarrollarnos y entender como las variables explicativas afectan a nuestra variable de interes: 'RainTomorrow', que es una varible binaria. Luego realizar predicciones utilizando modelos de regresion logistica con diferentes caracteristicas, y comparar cual tiene mejor comportameinto a la hora de predecir la presencia de lluvia para el dia siguiente dada ciertas condiciones del dia previo.\
<br>

# **Entorno**

Me gustaria meter aca directamente el dasaset preprocesado por la parte una del tp (hacer un csv del limpio y cargarlo directo aca). Pero tambien cargar distintas variantes del mismo, ya que la interpretacion del mismo puede diferir al tener todas variable snumiercas estandar (no saber a que ciudad se corresponde un numero, por ejemplo)

Yo cargaria el limpio final, y otro no estandarizado ni codificado ni reducido en clases, pero si imputado.

Este segundo mas con el fin de comparaciones de si las logistas van mejor con o sin codificaciones, y demas.

### Librerias

### Datos

# **Analisis Descriptivo**

Si bien esto ya estaba hecho, me gustaria enfocarlo ahora un poco mas en balances de clases, estudiar el comportamiento de presencia de lluvia con respecto a las otras variables, etc.

### Analisis

# **Procesamiento**

Si nos encontramos desbalances, o transformaciones que resultan ventajosas para la clasificacion, podemos agregarlo aca

### Transformacion

# **Predecciones**

Cuando consideremos que ya el dataset esta en su mejor forma, definimos y jugamos con los modelos

### Modelos

### Interpretaciones

Aca vamos a tener que meter toda la parte de explicabilidad para ver las poderaciones y cpmportamiento de los modelos a la hora de predecir y entrenar

# **Conclusiones**

Concluciones de mejores predicciones y los modelos